# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time 

import gmaps.datasets

#from ipywidgets.embed import embed_minimal_html 


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "../output_data/clean_city_data.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cabo San Lucas,22.89,-109.91,87.80,58,5,10.29,MX,1603501262
1,Kapaa,22.08,-159.32,82.00,78,90,12.75,US,1603501265
2,New Norfolk,-42.78,147.06,62.01,56,99,5.01,AU,1603501267
3,Puerto Ayora,-0.74,-90.35,69.01,92,52,1.01,EC,1603501257
4,Uvat,59.14,68.90,18.84,97,0,4.32,RU,1603501272
...,...,...,...,...,...,...,...,...,...
571,São Tomé,0.34,6.73,78.51,85,100,7.14,ST,1603502391
572,Iberia,29.95,-91.75,71.60,100,20,5.82,US,1603502630
573,Hobyo,5.35,48.53,81.07,77,100,11.01,SO,1603502557
574,Thaton,16.92,97.37,77.25,86,100,3.74,MM,1603502634


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [30]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [31]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Note:
> I included 2 screenshots of the heat maps in the images folder to make sure its the required angle.
***

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# # Filter by humidity below 80 percent
# humid_df = wind_df[wind_df["Humidity"] < 80]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,Cape Town,-33.93,18.42,73.40,60,0,9.17,ZA,1603461435
50,East London,-33.02,27.91,75.20,78,0,6.93,ZA,1603461490
140,Andros,37.83,24.93,73.00,54,0,4.00,GR,1603461623
252,Methóni,36.82,21.70,75.20,47,0,6.93,GR,1603461941
291,Waddān,29.16,16.14,79.99,21,0,7.43,LY,1603462026
330,Kruisfontein,-34.00,24.73,75.99,67,0,7.00,ZA,1603462111
357,Mahrīz,31.59,54.43,73.40,16,0,6.93,IR,1603462171
366,Panāgar,23.30,79.98,77.00,47,0,2.24,IN,1603462192
401,Liliani,32.21,72.95,75.72,20,0,4.45,PK,1603462269
497,Pilāni,28.37,75.60,79.57,17,0,6.58,IN,1603462482


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

Issues with finding a hotel at Waddān
Issues with finding a hotel at Kruisfontein
Issues with finding a hotel at Panāgar


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,Cape Town,-33.93,18.42,73.40,60,0,9.17,ZA,1603461435,Atlanticview Cape Town Boutique Hotel
50,East London,-33.02,27.91,75.20,78,0,6.93,ZA,1603461490,Premier Hotels & Resorts
140,Andros,37.83,24.93,73.00,54,0,4.00,GR,1603461623,DEOS Andros Villas
252,Methóni,36.82,21.70,75.20,47,0,6.93,GR,1603461941,ACHILLES HOTEL
291,Waddān,29.16,16.14,79.99,21,0,7.43,LY,1603462026,NA
330,Kruisfontein,-34.00,24.73,75.99,67,0,7.00,ZA,1603462111,NA
357,Mahrīz,31.59,54.43,73.40,16,0,6.93,IR,1603462171,مرکز اقامتی سیاحتی باغ پهلوانپور
366,Panāgar,23.30,79.98,77.00,47,0,2.24,IN,1603462192,NA
401,Liliani,32.21,72.95,75.72,20,0,4.45,PK,1603462269,Umer Tea Stall
497,Pilāni,28.37,75.60,79.57,17,0,6.58,IN,1603462482,NL Hotel


In [33]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,Cape Town,-33.93,18.42,73.40,60,0,9.17,ZA,1603461435,Atlanticview Cape Town Boutique Hotel
50,East London,-33.02,27.91,75.20,78,0,6.93,ZA,1603461490,Premier Hotels & Resorts
140,Andros,37.83,24.93,73.00,54,0,4.00,GR,1603461623,DEOS Andros Villas
252,Methóni,36.82,21.70,75.20,47,0,6.93,GR,1603461941,ACHILLES HOTEL
357,Mahrīz,31.59,54.43,73.40,16,0,6.93,IR,1603462171,مرکز اقامتی سیاحتی باغ پهلوانپور
401,Liliani,32.21,72.95,75.72,20,0,4.45,PK,1603462269,Umer Tea Stall
497,Pilāni,28.37,75.60,79.57,17,0,6.58,IN,1603462482,NL Hotel


In [34]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

7

In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Checking hotel info:
len(hotel_info)

7

In [39]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Note:
> I included 2 screenshots of the hotel maps in the images folder to make sure its the required angle. 
***